In [1]:
import backtrader as bt
import talib as ta
import pandas as pd
import json
from futu import *
import time
import numpy as np
import math
from typing import List
import datetime
import pandas as pd
import backtrader as bt
import backtrader.indicators as btind
import matplotlib
matplotlib.use('nbagg')  # 适用于 Jupyter 的交互式后端
import matplotlib.pyplot as plt  # 导入 matplotlib 确保显示
plt.rcParams['figure.figsize'] = (16, 12)
plt.rcParams['figure.dpi'] = 300

In [ ]:
from futu import *

def get_stock_pool(pool_name="全部"):
    quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
    ret, data = quote_ctx.get_user_security(pool_name)
    if ret == RET_OK:
        # print(data)
        if data.shape[0] > 0:  # 如果自选股列表不为空
            res = data['code'].values.tolist()
        else:
            res = []
    else:
        print('error:', data)
        res = []
    quote_ctx.close()
    return res

In [6]:
stock_pool = get_stock_pool()

2025-10-22 15:20:20,724 | 42768 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=3, host=127.0.0.1, port=11111, user_id=31759416
2025-10-22 15:20:20,790 | 42768 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=3


In [2]:
df = pd.read_parquet('future/data/kline_data.parquet')
df.head()
df.dtypes

code                     object
name                     object
time_key         datetime64[ns]
open                    float64
close                   float64
high                    float64
low                     float64
pe_ratio                float64
volume                    int64
turnover_rate           float64
turnover                float64
change_rate             float64
dtype: object

In [6]:
def get_all_plate(market: Market):
    quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
    ret, data = quote_ctx.get_plate_list(market, Plate.CONCEPT)
    res = {}
    if ret == RET_OK:
        for index, row in data.iterrows():
            res[row['plate_name']] = row['code']
    else:
        print('error:', data)
    quote_ctx.close()
    return res


In [7]:
all_plates = get_all_plate(Market.SH)

2025-10-24 16:57:20,057 | 39940 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=2, host=127.0.0.1, port=11111, user_id=31759416
2025-10-24 16:57:20,138 | 39940 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=2


In [19]:
need_plates_topic = ["稀土概念", "人工智能", "新能源汽车", "固态电池", "白酒", "银行保险证券", "芯片", "国产替代", "机器人"]

In [ ]:
res = []
for name, code in all_plates.items():
    res.append(name)
remp = ','.join(res)
print(len(res))

752


In [25]:
from openai import OpenAI

# for backward compatibility, you can still use `https://api.deepseek.com/v1` as `base_url`.
def get_ai_recommendation(prompt: str) -> str:
    client = OpenAI(api_key="sk-8c887e6454f741cc9553da2be62487af", base_url="https://api.deepseek.com")
    response = client.chat.completions.create(
        model="deepseek-chat",
            messages=[
                {"role": "system", "content": "你是一个专业的股票助手。"},
                {"role": "user", "content": prompt},
            ],
            max_tokens=1024,
        temperature=0.7,
        stream=False
    )
    return response.choices[0].message.content


In [38]:
prompt = f"""请从以下找出的概念板块中找出和列出的主题相关的概念板块，并以json格式输出
概念板块:
{remp}
主题:
{need_plates_topic}
输出示例：
```json{{
    "concept": [
        "稀土概念",
        "AI",
    ]
}}```
"""

In [39]:
plates = get_ai_recommendation(prompt)

In [42]:
import json_repair

In [43]:
decoded_object = json_repair.loads(plates)

In [102]:
decoded_object

{'concept': ['稀土永磁',
  '人工智能',
  '新能源车',
  '固态电池',
  '白酒概念',
  '券商',
  '保险概念',
  '银行概念',
  '芯片概念',
  '国产操作系统',
  '机器人概念']}

In [49]:
target_plate_codes = []
for item in decoded_object['concept']:
    target_plate_codes.append(all_plates[item])

In [81]:
# 交换 all_plates 字典的键与值
all_plates = {v: k for k, v in all_plates.items()}
target_plate_names = [all_plates[code] for code in target_plate_codes]

In [77]:
from typing import Dict
def filter_stocks_by_kdj_criteria(market: Market, plate_code: str) -> Dict[str, str]:
    """
    根据KDJ指标筛选股票
    
    参数:
        market: 市场类型，如 Market.SH, Market.SZ
        plate_code: 板块代码
    
    返回:
        Dict[str, str]: 股票代码到股票名称的映射字典
    """
    # 存储筛选结果
    stock_results = {}
    
    try:
        # 初始化行情上下文
        quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
        
        # 创建价格过滤器（价格范围2-1000）
        simple_filter = SimpleFilter()
        simple_filter.filter_min = 2
        simple_filter.filter_max = 1000
        simple_filter.stock_field = StockField.CUR_PRICE
        simple_filter.is_no_filter = True
        
        # 创建模式过滤器（KDJ低位金叉）
        # pattern_filter = PatternFilter()
        # pattern_filter.ktype = KLType.K_DAY
        # pattern_filter.stock_field = PatternField.PatternField_KDJGoldCrossLow
        # pattern_filter.is_no_filter = True
        
        # 创建自定义指标过滤器（KDJ的J值小于-5）
        custom_filter = CustomIndicatorFilter()
        custom_filter.ktype = KLType.K_DAY
        custom_filter.stock_field1 = StockField.KDJ_J
        custom_filter.stock_field1_para = [9, 3, 3]
        custom_filter.stock_field2 = StockField.VALUE
        # custom_filter.stock_field2_para = [9, 3, 3]
        custom_filter.relative_position = RelativePosition.LESS
        custom_filter.value = -5
        custom_filter.is_no_filter = False
        
        # 分页查询
        nBegin = 0
        last_page = False
        ret_list = list()
        
        while not last_page:
            nBegin += len(ret_list)
            # 使用传入的market和plate_code参数
            ret, ls = quote_ctx.get_stock_filter(
                market=market, 
                filter_list=[simple_filter, custom_filter], 
                begin=nBegin, 
                plate_code=plate_code
            )  
            
            if ret == RET_OK:
                last_page, all_count, ret_list = ls
                print(f'all count = {all_count}')
                
                # 收集股票代码和名称
                for item in ret_list:
                    stock_results[item.stock_code] = item.stock_name
            else:
                print(f'error: {ls}')
                break
            
            # 添加时间间隔，避免触发限频
            time.sleep(3)
            
    except Exception as e:
        print(f'筛选过程中发生错误: {e}')
    finally:
        # 确保关闭连接
        try:
            if 'quote_ctx' in locals():
                quote_ctx.close()
        except:
            pass
    
    return stock_results

In [ ]:
filtered_dict = {}
for plate_code in target_plate_codes:
    res = filter_stocks_by_kdj_criteria(Market.SH, plate_code)
    filtered_dict[plate_code] = [code for code, _ in res.items()]

In [89]:
def get_plate_stock(plate_code: str) -> List[str]:
    quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
    res = []
    ret, data = quote_ctx.get_plate_stock(plate_code)
    if ret == RET_OK:
        res = data['code'].values.tolist()
    else:
        print('error:', data)
        return []
    quote_ctx.close()
    return res

In [91]:
all_stock_dict = {}
for plate_code in target_plate_codes:
    all_stock_dict[plate_code] = get_plate_stock(plate_code)
    time.sleep(10)


2025-10-23 11:25:04,638 | 42768 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=48, host=127.0.0.1, port=11111, user_id=31759416
2025-10-23 11:25:14,660 | 42768 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=49, host=127.0.0.1, port=11111, user_id=31759416
2025-10-23 11:25:24,733 | 42768 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=50, host=127.0.0.1, port=11111, user_id=31759416
2025-10-23 11:25:34,801 | 42768 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=51, host=127.0.0.1, port=11111, user_id=31759416
2025-10-23 11:25:44,828 | 42768 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=52, host=127.0.0.1, port=11111, user_id=31759416
2025-10-23 11:25:54,858 | 42768 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=53, host=127.0.0.1, port=11111, user_id=31759416
2025-10-23 11:26:04,882 | 42768 | [open_contex

In [100]:
plate_codes = []
stock_cnts = []
fitered_stock_cnts = []
plate_name =[]
for plate_code, stock_codes in all_stock_dict.items():
    plate_codes.append(plate_code)
    plate_name.append(all_plates[plate_code])
    stock_cnts.append(len(stock_codes))
    fitered_stock_cnts.append(len(filtered_dict[plate_code]))
df = pd.DataFrame({
    '板块代码': plate_codes,
    '板块名称': plate_name,
    '股票数量': stock_cnts,
    '筛选后股票数量': fitered_stock_cnts
})
df['筛选后股票数量占比'] = df['筛选后股票数量'] / df['股票数量']
    

In [2]:
df = pd.read_parquet('future/data/kline_data.parquet')
print(df['code'].unique())

['SH.601138' 'SZ.000776' 'SH.688318' 'SH.688385' 'SH.600111' 'SZ.000034'
 'SH.688077' 'SH.688195' 'SZ.300657' 'SZ.002466' 'SH.601216' 'SH.688018'
 'SH.600580' 'SZ.002190' 'SH.600602' 'SZ.301219' 'HK.01810' 'SZ.301357'
 'SH.600570' 'SH.601888' 'SH.688017' 'SZ.301128' 'SZ.300499' 'SH.603893'
 'SZ.002475' 'SZ.300502' 'SH.600751' 'SH.688041' 'SZ.002085' 'SH.600031'
 'SZ.300274' 'SZ.300308' 'SH.601127' 'SZ.002156' 'SZ.002077' 'SH.600171'
 'SZ.000158' 'SH.600460' 'SZ.300750' 'SH.600036' 'SZ.002594' 'SZ.301236']


In [117]:
df = df[df['code'] == 'SH.600460']
df = df.sort_values(by='time_key')
df = compute_kdj(df)


In [3]:
df['time_key'].max()

Timestamp('2025-10-24 00:00:00')

In [1]:
import talib
import numpy as np
import pandas as pd
from future.Selector import BBIKDJSelector, SuperB1Selector

In [2]:
custom_selector = BBIKDJSelector(j_threshold=-5, bbi_min_window=30, max_window=120, price_range_pct=50,
bbi_q_threshold=0.5, j_q_threshold=0.3)
new_selector = SuperB1Selector(B1_params={'j_threshold': -5, 'bbi_min_window': 30, 'max_window': 120, 'price_range_pct': 50,
'bbi_q_threshold': 0.5, 'j_q_threshold': 0.3})


In [44]:
df = pd.read_parquet('future/data/kline_data.parquet')
df = df.rename(columns={'time_key': 'date'})
# df['BBI'] = compute_bbi(df)
stockes = df['code'].unique()
multi_stock_data = dict()
for stock_code in stockes:
    multi_stock_data[stock_code] = df[df['code'] == stock_code]


In [45]:
latest_date = df['date'].max()

In [48]:
selected_stocks = new_selector.select(latest_date, multi_stock_data)

Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Inde

d:\projects\stocks-reminding-system\future\Selector.py:332: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist["MA60"] = hist["close"].rolling(window=60, min_periods=1).mean()
d:\projects\stocks-reminding-system\future\Selector.py:332: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist["MA60"] = hist["close"].rolling(window=60, min_periods=1).mean()
d:\projects\stocks-reminding-system\future\Selector.py:348: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Inde

d:\projects\stocks-reminding-system\future\Selector.py:332: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist["MA60"] = hist["close"].rolling(window=60, min_periods=1).mean()
d:\projects\stocks-reminding-system\future\Selector.py:348: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist["DIF"] = compute_dif(hist)
d:\projects\stocks-reminding-system\future\Selector.py:332: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Inde

...

d:\projects\stocks-reminding-system\future\Selector.py:332: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist["MA60"] = hist["close"].rolling(window=60, min_periods=1).mean()
d:\projects\stocks-reminding-system\future\Selector.py:348: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist["DIF"] = compute_dif(hist)
d:\projects\stocks-reminding-system\future\Selector.py:332: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Inde

d:\projects\stocks-reminding-system\future\Selector.py:332: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist["MA60"] = hist["close"].rolling(window=60, min_periods=1).mean()
d:\projects\stocks-reminding-system\future\Selector.py:348: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist["DIF"] = compute_dif(hist)
d:\projects\stocks-reminding-system\future\Selector.py:332: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Index(['code', 'name', 'date', 'open', 'close', 'high', 'low', 'pe_ratio',
       'volume', 'turnover_rate', 'turnover', 'change_rate', 'BBI'],
      dtype='object')
Inde

d:\projects\stocks-reminding-system\future\Selector.py:332: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist["MA60"] = hist["close"].rolling(window=60, min_periods=1).mean()
d:\projects\stocks-reminding-system\future\Selector.py:332: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist["MA60"] = hist["close"].rolling(window=60, min_periods=1).mean()
d:\projects\stocks-reminding-system\future\Selector.py:332: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [49]:
selected_stocks

[]

In [67]:
df = pd.read_parquet('future/BBIKDJ_selector_experiment_results.parquet')

In [ ]:
df[df.stock_code.isnull()==False]

In [ ]:
from openai import OpenAI

# for backward compatibility, you can still use `https://api.deepseek.com/v1` as `base_url`.
# client = OpenAI(api_key="sk-8c887e6454f741cc9553da2be62487af", base_url="https://api.deepseek.com")
client = OpenAI(api_key="EMPTY", base_url="https://inference-fuyao-ppu-c3.xiaopeng.link/inference/default/infer-dongl8-qwen3-235b/v1")
response = client.chat.completions.create(
    model="Qwen3-235B",
    # model="deepseek-chat",
    messages=[
        {"role": "system", "content": "你是一个有用的助手"},
        {"role": "user", "content": "你是谁"},
  ],
    max_tokens=2048,
    temperature=0.7,
    stream=False
)

print(response.choices[0].message.content)